In [17]:
import numpy as np
from scipy.optimize import least_squares, leastsq
import scipy.stats as st
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

measured_n = np.array([80021.0, 208053.0, 211602.0, 106890.0, 29842.0, 4811.0, 460.0, 25.0, 2.0]) #, 
# measured_n = np.array([77303.0, 201021.0, 204408.0, 103187.0, 28761.0, 4658.0, 449.0, 24.0, 2.0])
#ideal_p = np.array([0.0061, 0.0608, 0.2272, 0.3460, 0.2476, 0.0906, 0.0190, 0.0024, 0.0002]) # Vorobiev, Dushin,...
ideal_p = np.array([0.00674, 0.05965, 0.22055, 0.35090, 0.25438, 0.08935, 0.01674, 0.00169, 0.00740]) # Zucker & Holden
averageNeutrons = 1.711177403507187
idealNeutrons = 3.13

kerns = 9

def gkern(kernlen=kerns, efficiency=0.55):
    K = [[0 for i in range(kernlen)] for j in range(kernlen)] 
    for i in range(kernlen):
        for j in range(kernlen):
            if i <= j:
                K[i][j] = (math.factorial(j) / (math.factorial(i) * math.factorial(j - i))) * efficiency**i * (1 - efficiency)**(j - i)
    return np.array(K)

n_count = measured_n.sum()
measured_p = measured_n / n_count
ideal_n = ideal_p * n_count

print("n_count: ", n_count)

apprEff = averageNeutrons / idealNeutrons

sigmaN = measured_n ** 0.5
for k in range(len(sigmaN)):
    if k > 1:
        sigmaN[k] *= k ** 0.5
sigmaN = sigmaN**(-1)
print("sigmaN^-1: ", sigmaN)

def function_from_efficiency_N(efficiency):
    return (n_count * gkern(kerns, efficiency[0]).dot(ideal_p) - measured_n)*(sigmaN)

def function_from_efficiency_P(efficiency):
    return (gkern(kerns, efficiency[0]).dot(ideal_p)) - measured_p

def predicted_from_efficiency_N(efficiency):
    return n_count * gkern(kerns, efficiency[0]).dot(ideal_p)

def predicted_from_efficiency_P(efficiency):
    return gkern(kerns, efficiency[0]).dot(ideal_p)


# predicted_optimal_p = function_from_efficiency([21.6])
# print(predicted_optimal_p)

#https://stackoverflow.com/questions/14854339/in-scipy-how-and-why-does-curve-fit-calculate-the-covariance-of-the-parameter-es/14857441#14857441
#https://stackoverflow.com/questions/42388139/how-to-compute-standard-deviation-errors-with-scipy-optimize-least-squares
#https://stackoverflow.com/questions/14581358/getting-standard-errors-on-fitted-parameters-using-the-optimize-leastsq-method-i/21844726#21844726


# least_squares
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html#r20fc1df64af7-jjmore
# print("least_squares")
# result = least_squares(function_from_efficiency, 0.5, method='lm',jac='2-point',max_nfev=1000000)
# print(result)
# J = result.jac
# cov = np.linalg.inv(J.T.dot(J))
# print("cov: ", cov)
# sigma = np.sqrt(np.diagonal(cov))
# print("sigma: ", sigma)
# print("")
# print("")

# optimal_eff = 0.57421387
# predicted_optimal_p = function_from_efficiency([optimal_eff])
# print('Predicted optimal probabilities: ' + repr(predicted_optimal_p))
# err = mean_squared_error(measured_p, predicted_optimal_p)
# print('MSE: ' + repr(err))
# print('RMSE: ' + repr(err ** 0.5))
# err_abs = mean_absolute_error(measured_p, predicted_optimal_p)
# print('MSE_abs: ' + repr(err_abs))
# print('Cov * MSE ' + repr((cov ** 0.5) * err))
# print('SEm ' + repr((cov ** 0.5) / (n_count ** 0.5)))

# ax = plt.subplot()
# x_array = np.arange(0.01, 0.99, 0.01)
# y_array = np.zeros(len(x_array))
# for i in range(len(x_array)):
#     y = mean_squared_error(measured_p, function_from_efficiency([x_array[i]]))
#     y_array[i] = y
# line, = plt.plot(x_array, y_array, lw=2)
# plt.ylim(0, 0.2)
# plt.show()



# leastsq
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.leastsq.html
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html
pfitN, pcovN, infodictN, errmsgN, successN = leastsq(function_from_efficiency_N, 0.5, full_output=1, epsfcn=0.0001)
pfitP, pcovP, infodictP, errmsgP, successP = leastsq(function_from_efficiency_P, 0.5, full_output=1, epsfcn=0.0001)
print("leastsq")
print("pcovN: ", pcovN)
print("infodictN: ", infodictN)
print("errmsgN: ", errmsgN)
print("successN: ", successN)

print("pcovP: ", pcovP)
print("infodictP: ", infodictP)
print("errmsgP: ", errmsgP)
print("successP: ", successP)

predicted_N = predicted_from_efficiency_N(pfitN)
predicted_P = predicted_from_efficiency_P(pfitP)
print("predicted_N: ", predicted_N)
print("predicted_P: ", predicted_P)
print("measured_N: ", measured_n)
print("measured_P: ", measured_p)

sum_N = 0
sum_P = 0
for i in range(len(measured_n)):
    sum_N += (predicted_N[i] - measured_n[i])**2
    sum_P += (predicted_P[i] - measured_p[i])**2
residuals_variance_N = sum_N/(len(predicted_N)-1)
residuals_variance_P = sum_P/(len(predicted_P)-1)
print("residuals_variance_N: ", residuals_variance_N)
print("residuals_variance_P: ", residuals_variance_P)

pcovN = pcovN # don't multiply to residuals sum, cov already with errors
pcovP = pcovP * residuals_variance_P
errorN = [] 
errorP = [] 
for i in range(len(pfitN)):
    try:
        errorN.append(np.absolute(pcovN[i][i])**0.5)
        errorP.append(np.absolute(pcovP[i][i])**0.5)
    except:
        errorN.append( 0.00 )
        errorP.append( 0.00 ) 

print("pfitP: ", pfitP)
print("perrP: ", np.array(errorP))
p_sigmaP = np.sqrt(np.diag(pcovP))
print("p_sigmaP: ", p_sigmaP)
print("")
print("!apprEff: ", apprEff)
print("!pfitN: ", pfitN)
print("!perrN: ", np.array(errorN))
p_sigmaN = np.sqrt(np.diag(pcovN))
print("!p_sigmaN: ", p_sigmaN)


n_count:  641706.0
sigmaN^-1:  [0.00353507 0.00219237 0.00153718 0.00176592 0.00289438 0.00644759
 0.01903467 0.07559289 0.25      ]
leastsq
pcovN:  [[2.86452401e-07]]
infodictN:  {'fvec': array([ 4.77298896,  2.55080231,  0.15275403,  0.66994133,  0.52900275,
        4.48565853, 11.67216763, 17.44728036,  8.42813558]), 'nfev': 11, 'fjac': array([[ 1.86841681e+03,  4.65279380e-01, -1.50205397e-01,
        -3.51579929e-01, -2.66701915e-01, -1.35945654e-01,
        -8.84632445e-02, -1.14736469e-01, -7.29597024e-02]]), 'ipvt': array([1], dtype=int32), 'qtf': array([0.02530063])}
errmsgN:  Both actual and predicted relative reductions in the sum of squares
  are at most 0.000000 and the relative error between two consecutive iterates is at 
  most 0.000000
successN:  3
pcovP:  [[0.81613851]]
infodictP:  {'fvec': array([1.21364112e-03, 8.97853155e-04, 5.85493295e-04, 1.45222521e-03,
       6.80869030e-04, 1.17451415e-03, 9.75499649e-04, 3.66137537e-04,
       5.37668520e-05]), 'nfev': 15, '